## Código utilizado para análise de dados da reportagem: [4,4 mil candidatos que se declararam negros voltam a se declarar brancos em 2024, quando verba para negros será menor](https://g1.globo.com/politica/eleicoes/2024/eleicao-em-numeros/noticia/2024/10/05/44-mil-candidatos-voltam-a-se-declarar-brancos-em-2024-quando-verba-para-negros-sera-menor.ghtml)

In [1]:
# Importa as bibliotecas necessárias para a análise
import pandas as pd
import numpy as np
import re
import numpy as np

In [2]:
# Me mostre tudo
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

## 📅 2016

In [4]:
df_2016 = pd.read_csv("data/consulta_cand_2016_BRASIL.csv", delimiter =';', encoding='latin-1')

C:\Users\judite\AppData\Local\Temp\ipykernel_1896\1729244441.py:1: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2016 = pd.read_csv("consulta_cand_2016_BRASIL.csv", delimiter =';', encoding='latin-1')


In [5]:
df_2016.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NM_TIPO_DESTINACAO_VOTOS,CD_SITUACAO_CANDIDATO_TOT,DS_SITUACAO_CANDIDATO_TOT,ST_PREST_CONTAS,ST_SUBSTITUIDO,SQ_SUBSTITUIDO,SQ_ORDEM_SUPLENCIA,DT_ACEITE_CANDIDATURA
0,13/03/2023,15:28:45,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,MA,7439,BURITI BRAVO,13,VEREADOR,100000004568,14456,MARIA SOLIDADE COSME DE OLIVEIRA,XUXA,#NULO#,95168559387,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,COLIGAÇÃO,14,PTB,PARTIDO TRABALHISTA BRASILEIRO,-1,#NULO#,#NULO#,#NULO#,100000000354,BURITI BRAVO NO RUMO CERTO I,PTB / DEM / PTN,1,BRASILEIRA NATA,MA,-3,PASSAGEM FRANCA,08/01/1943,73.0,35741881104,4,FEMININO,3,ENSINO FUNDAMENTAL INCOMPLETO,3,CASADO(A),3,PARDA,606,TRABALHADOR RURAL,10803.91,5,SUPLENTE,N,N,516812016,1604620166100044,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,9,12/08/2016 11:13:45
1,13/03/2023,15:28:45,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,PI,11231,MIGUEL ALVES,13,VEREADOR,180000010685,25000,FRANCINETE ALVES ALMEIDA,FRANCINETE,#NULO#,84681454320,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,COLIGAÇÃO,25,DEM,DEMOCRATAS,-1,#NULO#,#NULO#,#NULO#,180000000950,"O POVO QUER: NOVO TEMPO, NOVAS IDEIAS",PSOL / PP / PSDB / PR / PSD / PMN / PSL / PHS ...,1,BRASILEIRA NATA,PI,-3,MIGUEL ALVES,17/09/1979,37.0,24351441503,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),3,PARDA,265,PROFESSOR DE ENSINO FUNDAMENTAL,32499.97,5,SUPLENTE,N,S,434672016,969520166180017,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,9,16/08/2016 15:00:12
2,13/03/2023,15:28:45,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,PI,11231,MIGUEL ALVES,13,VEREADOR,180000010695,11000,IVANILDO SOUSA MOREIRA,PROFESSOR IVANILDO,#NULO#,53564901353,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,COLIGAÇÃO,11,PP,PARTIDO PROGRESSISTA,-1,#NULO#,#NULO#,#NULO#,180000000950,"O POVO QUER: NOVO TEMPO, NOVAS IDEIAS",PSOL / PP / PSDB / PR / PSD / PMN / PSL / PHS ...,1,BRASILEIRA NATA,PI,-3,MIGUEL ALVES,31/10/1974,42.0,27355661570,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),3,PARDA,265,PROFESSOR DE ENSINO FUNDAMENTAL,32499.97,5,SUPLENTE,N,N,434622016,917320166180017,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,11,16/08/2016 15:00:04
3,13/03/2023,15:28:45,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,PI,11231,MIGUEL ALVES,13,VEREADOR,180000010700,11333,INÁCIO ALVES DA SILVA,INÁCIO,#NULO#,75498189320,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,COLIGAÇÃO,11,PP,PARTIDO PROGRESSISTA,-1,#NULO#,#NULO#,#NULO#,180000000950,"O POVO QUER: NOVO TEMPO, NOVAS IDEIAS",PSOL / PP / PSDB / PR / PSD / PMN / PSL / PHS ...,1,BRASILEIRA NATA,PI,-3,MIGUEL ALVES,17/05/1977,39.0,23500221503,2,MASCULINO,5,ENSINO MÉDIO INCOMPLETO,1,SOLTEIRO(A),3,PARDA,999,OUTROS,32499.97,5,SUPLENTE,N,N,434662016,951320166180017,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,16,16/08/2016 15:00:10
4,13/03/2023,15:28:45,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10

### 🧹 Limpando duplicados

In [6]:
# checando os duplicados segundo o número de identificação do título de eleitor
duplicados = df_2016[df_2016.duplicated(subset='NR_TITULO_ELEITORAL_CANDIDATO', keep=False)]

In [7]:
df_2016 = df_2016.drop_duplicates(subset='NR_TITULO_ELEITORAL_CANDIDATO')

In [8]:
df_2016["NR_TITULO_ELEITORAL_CANDIDATO"].nunique()

496415

### 🆗 Filtrando aptos

In [9]:
df_2016 = df_2016[df_2016["CD_SITUACAO_CANDIDATURA"] == 12]

### 🧩 Padronizando o DF

In [10]:
colunas_selecionadas = ['NR_TITULO_ELEITORAL_CANDIDATO', 'DS_COR_RACA', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_CARGO', 'DS_SITUACAO_CANDIDATURA', 'SG_PARTIDO', 'SG_UF', 'NM_UE']
df_2016 = df_2016[colunas_selecionadas]
df_2016.columns = [f"{coluna}_2016" for coluna in df_2016.columns]

In [11]:
df_2016.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2016,DS_COR_RACA_2016,NM_CANDIDATO_2016,NM_URNA_CANDIDATO_2016,DS_CARGO_2016,DS_SITUACAO_CANDIDATURA_2016,SG_PARTIDO_2016,SG_UF_2016,NM_UE_2016
0,35741881104,PARDA,MARIA SOLIDADE COSME DE OLIVEIRA,XUXA,VEREADOR,APTO,PTB,MA,BURITI BRAVO
1,24351441503,PARDA,FRANCINETE ALVES ALMEIDA,FRANCINETE,VEREADOR,APTO,DEM,PI,MIGUEL ALVES
2,27355661570,PARDA,IVANILDO SOUSA MOREIRA,PROFESSOR IVANILDO,VEREADOR,APTO,PP,PI,MIGUEL ALVES
3,23500221503,PARDA,INÁCIO ALVES DA SILVA,INÁCIO,VEREADOR,APTO,PP,PI,MIGUEL ALVES
5,41891390922,PARDA,RONALDO BELLO DAS CHAGAS,RONALDO CHAGAS,VEREADOR,APTO,PMDB,SC,IBICARÉ


## 📅 2020

In [12]:
df_2020 = pd.read_csv("data/consulta_cand_2020_BRASIL.csv", delimiter =';', encoding='latin-1')

In [13]:
df_2020.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NM_TIPO_DESTINACAO_VOTOS,CD_SITUACAO_CANDIDATO_TOT,DS_SITUACAO_CANDIDATO_TOT,ST_PREST_CONTAS
0,02/05/2024,00:33:52,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,BA,34592,COARACI,11,PREFEITO,50001251223,13,JOSEFINA MARIA CASTRO DOS SANTOS,JOSEFINA CASTRO,#NULO#,8348235515,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,COLIGAÇÃO,13,PT,PARTIDO DOS TRABALHADORES,-1,#NULO#,#NULO#,#NULO#,50000155260,A FORÇA DA UNIÃO,PSD / PROS / PSB / PT,1,BRASILEIRA NATA,BA,-3,ILHÉUS,21/08/1951,69.0,16854590540,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),2,PRETA,142,PROFESSOR DE ENSINO SUPERIOR,123077.42,4,NÃO ELEITO,N,S,-1,6003559620206050135,2,DEFERIDO,16,DEFERIDO COM RECURSO,SIM,Válido,2,Deferido,S
1,02/05/2024,00:33:52,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,PR,74420,NOVO ITACOLOMI,11,PREFEITO,160000773419,55,MOACIR ANDREOLLA,MOACIR,#NULO#,64465160968,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,COLIGAÇÃO,55,PSD,PARTIDO SOCIAL DEMOCRÁTICO,-1,#NULO#,#NULO#,#NULO#,160000074895,RUMO CERTO AO FUTURO,PSD / PP / PSL,1,BRASILEIRA NATA,RS,-3,ERECHIM,01/07/1966,54.0,35542450647,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRANCA,275,PREFEITO,123077.42,1,ELEITO,N,S,-1,6003032720206160028,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S
2,02/05/2024,00:33:52,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,PA,5231,SALINÓPOLIS,13,VEREADOR,140000641433,12345,JOÃO VICTOR DE BORGES CORREA,VICTOR FAROL,#NULO#,71324038268,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,-1,#NULO#,#NULO#,#NULO#,140000056321,PARTIDO ISOLADO,PDT,1,BRASILEIRA NATA,PA,-3,SALINÓPOLIS,01/03/1982,38.0,43353371317,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),3,PARDA,999,OUTROS,34729.59,5,SUPLENTE,N,S,-1,6001321620206140064,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S
3,02/05/2024,00:33:52,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,71072,SÃO PAULO,13,VEREADOR,250001206039,70078,MARIA DAS GRAÇAS DA SILVA LOPES,GRAÇA LOPES,#NULO#,1191949885,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,70,AVANTE,AVANTE,-1,#NULO#,#NULO#,#NULO#,250000148339,PARTIDO ISOLADO,AVANTE,1,BRASILEIRA NATA,PB,-3,UMBUZEIRO,26/11/1960,60.0,798110183,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,999,OUTROS,3675197.12,4,NÃO ELEITO,N,S,-1,6020510220206260001,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S
4,02/05/2024,00:33:52,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,PR,75213,CONTENDA,13,VEREADOR,160000659785,17333,LEANDRO DOS PRAZERES GONÇALVES,LEANDRO GONÇALVES COBRADOR,#NULO#,5073603974,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,17,PSL,PARTIDO SOCIAL LIBERAL,-1,#NULO#,#NULO#,#NULO#,160000058855,PARTIDO ISOLADO,PSL,1,BRASILEIRA NATA,PR,-3,CURITIBA,13/08/1983,37.0,80291980680,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRANCA,160,COBRADOR DE TRANSPO

### 🧹 Limpando duplicados

In [14]:
# checando os duplicados segundo o número de identificação do título de eleitor
duplicados = df_2020[df_2020.duplicated(subset='NR_TITULO_ELEITORAL_CANDIDATO', keep=False)]

In [15]:
df_2020 = df_2020.drop_duplicates(subset='NR_TITULO_ELEITORAL_CANDIDATO')

In [16]:
df_2020["NR_TITULO_ELEITORAL_CANDIDATO"].nunique()

556498

### 🆗 Filtrando aptos

In [17]:
df_2020 = df_2020[df_2020["CD_SITUACAO_CANDIDATURA"] == 12]

### 🧩 Padronizando o DF

In [18]:
colunas_selecionadas = ['NR_TITULO_ELEITORAL_CANDIDATO', 'DS_COR_RACA', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_CARGO', 'DS_SITUACAO_CANDIDATURA', 'SG_PARTIDO', 'SG_UF', 'NM_UE']
df_2020 = df_2020[colunas_selecionadas]
df_2020.columns = [f"{coluna}_2020" for coluna in df_2020.columns]

In [19]:
df_2020.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2020,DS_COR_RACA_2020,NM_CANDIDATO_2020,NM_URNA_CANDIDATO_2020,DS_CARGO_2020,DS_SITUACAO_CANDIDATURA_2020,SG_PARTIDO_2020,SG_UF_2020,NM_UE_2020
0,16854590540,PRETA,JOSEFINA MARIA CASTRO DOS SANTOS,JOSEFINA CASTRO,PREFEITO,APTO,PT,BA,COARACI
1,35542450647,BRANCA,MOACIR ANDREOLLA,MOACIR,PREFEITO,APTO,PSD,PR,NOVO ITACOLOMI
2,43353371317,PARDA,JOÃO VICTOR DE BORGES CORREA,VICTOR FAROL,VEREADOR,APTO,PDT,PA,SALINÓPOLIS
3,798110183,BRANCA,MARIA DAS GRAÇAS DA SILVA LOPES,GRAÇA LOPES,VEREADOR,APTO,AVANTE,SP,SÃO PAULO
4,80291980680,BRANCA,LEANDRO DOS PRAZERES GONÇALVES,LEANDRO GONÇALVES COBRADOR,VEREADOR,APTO,PSL,PR,CONTENDA


## 📅 2024

In [20]:
df_2024 = pd.read_csv("data/consulta_cand_2024_BRASIL.csv", delimiter =';', encoding='latin-1')

In [21]:
df_2024.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,DS_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,SG_UF_NASCIMENTO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
0,17/09/2024,16:31:28,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,BA,33782,BOM JESUS DA SERRA,13,VEREADOR,50002239886,20357,MARIA ROCHA DIAS,MARY DIAS,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,20,PODE,PODEMOS,-1,#NULO#,#NULO#,#NULO#,50001744185,PARTIDO ISOLADO,PODE,BA,12/11/1985,106637890540,4,FEMININO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),1,BRANCA,999,OUTROS,-1,#NULO#
1,17/09/2024,16:31:28,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,BA,35815,IPIRÁ,13,VEREADOR,50002239960,36366,EDMILSON ARAUJO DA PAZ,MESTRE EDMILSON,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,36,AGIR,AGIR,-1,#NULO#,#NULO#,#NULO#,50001744200,PARTIDO ISOLADO,AGIR,BA,03/10/1976,108334920590,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,1,SOLTEIRO(A),2,PRETA,999,OUTROS,-1,#NULO#
2,17/09/2024,16:31:28,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,BA,38253,RIACHÃO DAS NEVES,13,VEREADOR,50002239560,12444,VERILMA LIMA DOS SANTOS,VERINHA,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,-1,#NULO#,#NULO#,#NULO#,50001744130,PARTIDO ISOLADO,PDT,BA,04/12/1980,101448370566,4,FEMININO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),2,PRETA,402,"VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIA...",-1,#NULO#
3,17/09/2024,16:31:28,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,BA,34711,CONDE,13,VEREADOR,50002239720,25610,JORGE CARDOSO DE ALMEIDA,JORGE DA ILHA,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,25,PRD,PARTIDO RENOVAÇÃO DEMOCRÁTICA,-1,#NULO#,#NULO#,#NULO#,50001744156,PARTIDO ISOLADO,PRD,BA,30/03/1974,62709510582,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,1,SOLTEIRO(A),2,PRETA,999,OUTROS,-1,#NULO#
4,17/09/2024,16:31:28,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,BA,33910,BRUMADO,13,VEREADOR,50002301448,40321,MARCIA SOUZA MACHADO GOMES,MARCIA MACHADO,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,-1,#NULO#,#NULO#,#NULO#,50001744137,PARTIDO ISOLADO,PSB,BA,31/10/1969,53326730531,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,143,PROFESSOR DE ENSINO DE PRIMEIRO E SEGUNDO GRAUS,-1,#NULO#


### 🧹 Limpando duplicados

In [22]:
# checando os duplicados segundo o número de identificação do título de eleitor
duplicados = df_2024[df_2024.duplicated(subset='NR_TITULO_ELEITORAL_CANDIDATO', keep=False)]

In [23]:
df_2024 = df_2024.drop_duplicates(subset='NR_TITULO_ELEITORAL_CANDIDATO')

In [24]:
df_2024["NR_TITULO_ELEITORAL_CANDIDATO"].nunique()

462590

### 🧩 Padronizando o DF

In [25]:
colunas_selecionadas = ['NR_TITULO_ELEITORAL_CANDIDATO', 'DS_COR_RACA', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_CARGO', 'DS_SITUACAO_CANDIDATURA', 'SG_PARTIDO', 'SG_UF', 'NM_UE']
df_2024 = df_2024[colunas_selecionadas]
df_2024.columns = [f"{coluna}_2024" for coluna in df_2024.columns]

In [26]:
df_2024.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2024,DS_COR_RACA_2024,NM_CANDIDATO_2024,NM_URNA_CANDIDATO_2024,DS_CARGO_2024,DS_SITUACAO_CANDIDATURA_2024,SG_PARTIDO_2024,SG_UF_2024,NM_UE_2024
0,106637890540,BRANCA,MARIA ROCHA DIAS,MARY DIAS,VEREADOR,#NE,PODE,BA,BOM JESUS DA SERRA
1,108334920590,PRETA,EDMILSON ARAUJO DA PAZ,MESTRE EDMILSON,VEREADOR,#NE,AGIR,BA,IPIRÁ
2,101448370566,PRETA,VERILMA LIMA DOS SANTOS,VERINHA,VEREADOR,#NE,PDT,BA,RIACHÃO DAS NEVES
3,62709510582,PRETA,JORGE CARDOSO DE ALMEIDA,JORGE DA ILHA,VEREADOR,#NE,PRD,BA,CONDE
4,53326730531,BRANCA,MARCIA SOUZA MACHADO GOMES,MARCIA MACHADO,VEREADOR,#NE,PSB,BA,BRUMADO


## 🔗 Unindo DFs de anos diferentes

In [27]:
# Juntando df_2016 e df_2020 com base nas colunas corretas
df_raça_part1 = pd.merge(df_2016, df_2020, left_on='NR_TITULO_ELEITORAL_CANDIDATO_2016', right_on='NR_TITULO_ELEITORAL_CANDIDATO_2020', how='inner')

# Agora, juntando o resultado com df_2024
df_raça = pd.merge(df_raça_part1, df_2024, left_on='NR_TITULO_ELEITORAL_CANDIDATO_2016', right_on='NR_TITULO_ELEITORAL_CANDIDATO_2024', how='inner')

In [28]:
df_raça.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2016,DS_COR_RACA_2016,NM_CANDIDATO_2016,NM_URNA_CANDIDATO_2016,DS_CARGO_2016,DS_SITUACAO_CANDIDATURA_2016,SG_PARTIDO_2016,SG_UF_2016,NM_UE_2016,NR_TITULO_ELEITORAL_CANDIDATO_2020,DS_COR_RACA_2020,NM_CANDIDATO_2020,NM_URNA_CANDIDATO_2020,DS_CARGO_2020,DS_SITUACAO_CANDIDATURA_2020,SG_PARTIDO_2020,SG_UF_2020,NM_UE_2020,NR_TITULO_ELEITORAL_CANDIDATO_2024,DS_COR_RACA_2024,NM_CANDIDATO_2024,NM_URNA_CANDIDATO_2024,DS_CARGO_2024,DS_SITUACAO_CANDIDATURA_2024,SG_PARTIDO_2024,SG_UF_2024,NM_UE_2024
0,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,APTO,PTB,MG,JEQUITINHONHA,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,APTO,PSD,MG,JEQUITINHONHA,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,#NE,PSDB,MG,JEQUITINHONHA
1,3919891074,BRANCA,CLÁUDIO LÚCIO RODRIGUES,CACÁ LÚCIO,VEREADOR,APTO,PRB,GO,CATALÃO,3919891074,PARDA,CLAUDIO LUCIO RODRIGUES,KAKA DOS ANIMAIS,VEREADOR,APTO,REPUBLICANOS,GO,CATALÃO,3919891074,BRANCA,CLAUDIO LUCIO RODRIGUES,KAKA AMIGO DOS ANIMAIS,VEREADOR,#NE,PSDB,GO,CATALÃO
2,20926432704,PARDA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,APTO,PSB,PA,SANTA MARIA DAS BARREIRAS,20926432704,PRETA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,APTO,PSB,PA,SANTA MARIA DAS BARREIRAS,20926432704,PRETA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,#NE,PL,PA,SANTA MARIA DAS BARREIRAS
3,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIM,VEREADOR,APTO,PTN,BA,JUCURUÇU,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIN,VEREADOR,APTO,PSD,BA,JUCURUÇU,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIM DE ITAMARATY,VEREADOR,#NE,PSDB,BA,JUCURUÇU
4,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSA MARIA DA COSTA,VEREADOR,APTO,PT,MG,UBÁ,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSINHA DO LACTÁRIO,VEREADOR,APTO,PDT,MG,UBÁ,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSINHA DO LACTARIO,VEREADOR,#NE,MOBILIZA,MG,UBÁ


In [29]:
# Definindo as condições
condicoes = [
    (df_raça['DS_COR_RACA_2016'] != df_raça['DS_COR_RACA_2020'])
]

# Definindo os valores correspondentes
valores = ["mudou 2016 para 2020"]

# Criando a nova coluna com base nas condições
df_raça['Resultado'] = np.select(condicoes, valores, default="")

In [30]:
df_raça.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2016,DS_COR_RACA_2016,NM_CANDIDATO_2016,NM_URNA_CANDIDATO_2016,DS_CARGO_2016,DS_SITUACAO_CANDIDATURA_2016,SG_PARTIDO_2016,SG_UF_2016,NM_UE_2016,NR_TITULO_ELEITORAL_CANDIDATO_2020,DS_COR_RACA_2020,NM_CANDIDATO_2020,NM_URNA_CANDIDATO_2020,DS_CARGO_2020,DS_SITUACAO_CANDIDATURA_2020,SG_PARTIDO_2020,SG_UF_2020,NM_UE_2020,NR_TITULO_ELEITORAL_CANDIDATO_2024,DS_COR_RACA_2024,NM_CANDIDATO_2024,NM_URNA_CANDIDATO_2024,DS_CARGO_2024,DS_SITUACAO_CANDIDATURA_2024,SG_PARTIDO_2024,SG_UF_2024,NM_UE_2024,Resultado
0,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,APTO,PTB,MG,JEQUITINHONHA,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,APTO,PSD,MG,JEQUITINHONHA,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,#NE,PSDB,MG,JEQUITINHONHA,
1,3919891074,BRANCA,CLÁUDIO LÚCIO RODRIGUES,CACÁ LÚCIO,VEREADOR,APTO,PRB,GO,CATALÃO,3919891074,PARDA,CLAUDIO LUCIO RODRIGUES,KAKA DOS ANIMAIS,VEREADOR,APTO,REPUBLICANOS,GO,CATALÃO,3919891074,BRANCA,CLAUDIO LUCIO RODRIGUES,KAKA AMIGO DOS ANIMAIS,VEREADOR,#NE,PSDB,GO,CATALÃO,mudou 2016 para 2020
2,20926432704,PARDA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,APTO,PSB,PA,SANTA MARIA DAS BARREIRAS,20926432704,PRETA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,APTO,PSB,PA,SANTA MARIA DAS BARREIRAS,20926432704,PRETA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,#NE,PL,PA,SANTA MARIA DAS BARREIRAS,mudou 2016 para 2020
3,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIM,VEREADOR,APTO,PTN,BA,JUCURUÇU,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIN,VEREADOR,APTO,PSD,BA,JUCURUÇU,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIM DE ITAMARATY,VEREADOR,#NE,PSDB,BA,JUCURUÇU,
4,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSA MARIA DA COSTA,VEREADOR,APTO,PT,MG,UBÁ,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSINHA DO LACTÁRIO,VEREADOR,APTO,PDT,MG,UBÁ,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSINHA DO LACTARIO,VEREADOR,#NE,MOBILIZA,MG,UBÁ,


In [31]:
df_raça.to_csv('data/mudaram_2016_2020.csv', decimal=";", index=False, encoding='latin-1')

In [32]:
# Definindo as condições, que foram modificadas segundo a pauta
condicoes = [
    #(df_raça['DS_COR_RACA_2016'] != df_raça['DS_COR_RACA_2020']) & (df_raça['DS_COR_RACA_2016'] == df_raça['DS_COR_RACA_2024']),
   # (df_raça['DS_COR_RACA_2016'] != df_raça['DS_COR_RACA_2020']) & (df_raça['DS_COR_RACA_2016'] != df_raça['DS_COR_RACA_2024']) & (df_raça['DS_COR_RACA_2020'] != df_raça['DS_COR_RACA_2024']),
    (df_raça['DS_COR_RACA_2020'] != df_raça['DS_COR_RACA_2024'])
]

# Definindo os valores correspondentes
valores = [
   # "2016 e 2024 igual",
   # "cores diferentes todos os anos",
    "mudou de 2020 para 2024"
]

# Criando a nova coluna com base nas condições
df_raça['Resultado'] = np.select(condicoes, valores, default="")

In [33]:
df_raça.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2016,DS_COR_RACA_2016,NM_CANDIDATO_2016,NM_URNA_CANDIDATO_2016,DS_CARGO_2016,DS_SITUACAO_CANDIDATURA_2016,SG_PARTIDO_2016,SG_UF_2016,NM_UE_2016,NR_TITULO_ELEITORAL_CANDIDATO_2020,DS_COR_RACA_2020,NM_CANDIDATO_2020,NM_URNA_CANDIDATO_2020,DS_CARGO_2020,DS_SITUACAO_CANDIDATURA_2020,SG_PARTIDO_2020,SG_UF_2020,NM_UE_2020,NR_TITULO_ELEITORAL_CANDIDATO_2024,DS_COR_RACA_2024,NM_CANDIDATO_2024,NM_URNA_CANDIDATO_2024,DS_CARGO_2024,DS_SITUACAO_CANDIDATURA_2024,SG_PARTIDO_2024,SG_UF_2024,NM_UE_2024,Resultado
0,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,APTO,PTB,MG,JEQUITINHONHA,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,APTO,PSD,MG,JEQUITINHONHA,47554410205,PARDA,EDIVALDO RODRIGUES DOS SANTOS,EDIVALDO,VEREADOR,#NE,PSDB,MG,JEQUITINHONHA,
1,3919891074,BRANCA,CLÁUDIO LÚCIO RODRIGUES,CACÁ LÚCIO,VEREADOR,APTO,PRB,GO,CATALÃO,3919891074,PARDA,CLAUDIO LUCIO RODRIGUES,KAKA DOS ANIMAIS,VEREADOR,APTO,REPUBLICANOS,GO,CATALÃO,3919891074,BRANCA,CLAUDIO LUCIO RODRIGUES,KAKA AMIGO DOS ANIMAIS,VEREADOR,#NE,PSDB,GO,CATALÃO,mudou de 2020 para 2024
2,20926432704,PARDA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,APTO,PSB,PA,SANTA MARIA DAS BARREIRAS,20926432704,PRETA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,APTO,PSB,PA,SANTA MARIA DAS BARREIRAS,20926432704,PRETA,JOSE XAVIER DE OLIVEIRA,REIS,VEREADOR,#NE,PL,PA,SANTA MARIA DAS BARREIRAS,
3,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIM,VEREADOR,APTO,PTN,BA,JUCURUÇU,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIN,VEREADOR,APTO,PSD,BA,JUCURUÇU,81735530272,PARDA,ANORINO SOUZA SANTOS,MIZIM DE ITAMARATY,VEREADOR,#NE,PSDB,BA,JUCURUÇU,
4,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSA MARIA DA COSTA,VEREADOR,APTO,PT,MG,UBÁ,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSINHA DO LACTÁRIO,VEREADOR,APTO,PDT,MG,UBÁ,4952270281,BRANCA,CLEUSA MARIA DA COSTA,CLEUSINHA DO LACTARIO,VEREADOR,#NE,MOBILIZA,MG,UBÁ,


In [34]:
# Filtrando o DataFrame para manter apenas as linhas onde Resultado_Cores não está vazio
df_filtrado = df_raça[df_raça['Resultado'] != ""]

In [35]:
df_filtrado.head()

,NR_TITULO_ELEITORAL_CANDIDATO_2016,DS_COR_RACA_2016,NM_CANDIDATO_2016,NM_URNA_CANDIDATO_2016,DS_CARGO_2016,DS_SITUACAO_CANDIDATURA_2016,SG_PARTIDO_2016,SG_UF_2016,NM_UE_2016,NR_TITULO_ELEITORAL_CANDIDATO_2020,DS_COR_RACA_2020,NM_CANDIDATO_2020,NM_URNA_CANDIDATO_2020,DS_CARGO_2020,DS_SITUACAO_CANDIDATURA_2020,SG_PARTIDO_2020,SG_UF_2020,NM_UE_2020,NR_TITULO_ELEITORAL_CANDIDATO_2024,DS_COR_RACA_2024,NM_CANDIDATO_2024,NM_URNA_CANDIDATO_2024,DS_CARGO_2024,DS_SITUACAO_CANDIDATURA_2024,SG_PARTIDO_2024,SG_UF_2024,NM_UE_2024,Resultado
1,3919891074,BRANCA,CLÁUDIO LÚCIO RODRIGUES,CACÁ LÚCIO,VEREADOR,APTO,PRB,GO,CATALÃO,3919891074,PARDA,CLAUDIO LUCIO RODRIGUES,KAKA DOS ANIMAIS,VEREADOR,APTO,REPUBLICANOS,GO,CATALÃO,3919891074,BRANCA,CLAUDIO LUCIO RODRIGUES,KAKA AMIGO DOS ANIMAIS,VEREADOR,#NE,PSDB,GO,CATALÃO,mudou de 2020 para 2024
8,41324280850,BRANCA,ELIONILDA JOSÉ DE SANTANA,IRMÃ ELIONILDA,VICE-PREFEITO,APTO,PP,PE,ITAPISSUMA,41324280850,PARDA,ELIONILDA JOSÉ DE SANTANA,IRMÃ ELIONILDA,VEREADOR,APTO,MDB,PE,ITAPISSUMA,41324280850,BRANCA,ELIONILDA JOSE DE SANTANA,IRMÃ ELIONILDA,VEREADOR,#NE,PSB,PE,ITAPISSUMA,mudou de 2020 para 2024
9,122129830213,PRETA,THARCILA CRISTINA DA SILVA,THARCILA,VEREADOR,APTO,PSL,MG,CLÁUDIO,122129830213,PRETA,THARCILA CRISTINA DA SILVA,THARCILA DO SÃO FRANCISCO,VEREADOR,APTO,PODE,MG,CLÁUDIO,122129830213,PARDA,THARCILA CRISTINA DA SILVA,THARCILA,VICE-PREFEITO,#NE,PODE,MG,CLÁUDIO,mudou de 2020 para 2024
13,21263391767,PARDA,PAULO GOMES DOS SANTOS,PAULO GOMES,VEREADOR,APTO,PDT,SP,TAQUARAL,21263391767,BRANCA,PAULO GOMES DOS SANTOS,PAULO GOMES,VEREADOR,APTO,REPUBLICANOS,SP,TAQUARAL,21263391767,PARDA,PAULO GOMES DOS SANTOS,PAULO GOMES,VEREADOR,#NE,MDB,SP,TAQUARAL,mudou de 2020 para 2024
17,181437430213,BRANCA,LORENA SANTOS OLIVEIRA,LORENA SANTOS,VEREADOR,APTO,PMDB,MG,UBERLÂNDIA,181437430213,BRANCA,LORENA SANTOS OLIVEIRA,LORENA OLIVEIRA,VEREADOR,APTO,MDB,MG,UBERLÂNDIA,181437430213,PARDA,LORENA SANTOS OLIVEIRA,LORENA OLIVEIRA,VEREADOR,#NE,MDB,MG,UBERLÂNDIA,mudou de 2020 para 2024


In [36]:
df_filtrado.to_csv('data/cand_mudaram_declaracao.csv', decimal=";", index=False, encoding='latin-1')